In [5]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random

In [7]:
x = random.uniform(0, 15)
x

0.7095694997691593

In [74]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./vector_store/")
for collection in chroma_client.list_collections():
    chroma_client.delete_collection(collection.name)
collection = chroma_client.create_collection(
    name="object_embeddings",
    metadata={"hnsw:space": "cosine"},
)


In [75]:
embeddings_path = "./embeddings/"
for embedding_file in os.listdir(embeddings_path):
    embedding = np.load(os.path.join(embeddings_path, embedding_file))
    collection.add(
        documents=[embedding_file],
        embeddings=[embedding],
        metadatas=[
            {
                "filename": embedding_file,
                "object_id": embedding_file[:-4],
                "xc": random.uniform(0,15),
                "yc": random.uniform(0,15),
                "zc": random.uniform(0,15),
            }
        ],
        ids=[embedding_file[:-4]]
    )

In [76]:
import clip
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [77]:
text = clip.tokenize(["a red ball"]).to(device)

with torch.no_grad():
    text_features = model.encode_text(text).cpu().numpy().flatten()

In [78]:
results = collection.query(
    query_embeddings=text_features,
    n_results=3,
)

In [79]:
results

{'ids': [['red_ball', 'scissors', 'blue_chair']],
 'embeddings': None,
 'documents': [['red_ball.npy', 'scissors.npy', 'blue_chair.npy']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'xc': 14.239078869648552,
    'zc': 2.9384762503588973,
    'filename': 'red_ball.npy',
    'object_id': 'red_ball',
    'yc': 3.670390148138005},
   {'xc': 2.8511296705573907,
    'zc': 2.6308741928525836,
    'filename': 'scissors.npy',
    'object_id': 'scissors',
    'yc': 14.44808030086081},
   {'yc': 14.174496922965384,
    'zc': 14.898519920408086,
    'filename': 'blue_chair.npy',
    'object_id': 'blue_chair',
    'xc': 13.575146694229602}]],
 'distances': [[0.6693223714828491, 0.777964174747467, 0.8080158233642578]]}

In [73]:
results['metadatas'][0]

[{'filename': 'red_ball.npy',
  'xc': 13.296815562957082,
  'yc': 14.044838087413792,
  'object_id': 'red_ball',
  'zc': 7.353423644890979},
 {'xc': 9.117027952791387,
  'zc': 2.8492920993023185,
  'filename': 'scissors.npy',
  'yc': 9.969297068500294,
  'object_id': 'scissors'},
 {'yc': 12.08143950449802,
  'xc': 8.034867064120276,
  'filename': 'blue_chair.npy',
  'object_id': 'blue_chair',
  'zc': 6.919447084376897}]